In [1]:
!pip install scikit-multilearn

# Summary
This notebook explores the use of DistilBERT for two tasks:

- Multi-label emotion classification on the GoEmotions dataset

- Binary suicide risk detection on SuicideWatch posts

The objective is to train lightweight wrapper models that can classify input texts and return structured system prompts — as a backend for potential chatbot integration.

# Key Learning
- Transformer-based models like BERT require a different workflow than traditional ML or TensorFlow-based pipelines.

- Multi-label classification (vs. multiclass) introduces unique modeling and evaluation challenges — accuracy is misleading, and metrics like F1-micro/macro are more representative.

- Pretrained models are powerful but need careful handling when adapting to specialized or emotionally nuanced tasks.

# What I build
✅ DistilBERT GoEmotions Model

- Task: Multi-label classification of 28 emotions

- Performance: ~0.30 F1-micro/macro

- Purpose: Map chat inputs to nuanced emotional labels

✅ DistilBERT SuicideWatch Model

- Task: Binary classification (at-risk vs not)

- Performance: ~95% accuracy, precision, recall, and F1

- Purpose: High-sensitivity detection for mental health risk

✅ Optimized for deployment

- Applied pruning and quantization to reduce model size and inference latency

- Intended for integration as a preprocessor module in LLM-powered systems



In [2]:
import numpy as np 
import pandas as pd 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, precision_score, recall_score
import transformers
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

2025-07-10 09:45:10.521983: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752140710.885271      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752140710.989497      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/suicide-watch/Suicide_Detection.csv
/kaggle/input/go-emotions-google-emotions-dataset/go_emotions_dataset.csv
Using device: cuda


In [3]:
go_df = pd.read_csv('/kaggle/input/go-emotions-google-emotions-dataset/go_emotions_dataset.csv')
sw_df = pd.read_csv('/kaggle/input/suicide-watch/Suicide_Detection.csv')

print("GoEmotions Dataset by Google!")
go_df.info()
go_df.head()

print("\n\nSuicide Watch Dataset By Nikhileswar Komati!")
sw_df.info()
sw_df.head()

GoEmotions Dataset by Google!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211225 entries, 0 to 211224
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    211225 non-null  object
 1   text                  211225 non-null  object
 2   example_very_unclear  211225 non-null  bool  
 3   admiration            211225 non-null  int64 
 4   amusement             211225 non-null  int64 
 5   anger                 211225 non-null  int64 
 6   annoyance             211225 non-null  int64 
 7   approval              211225 non-null  int64 
 8   caring                211225 non-null  int64 
 9   confusion             211225 non-null  int64 
 10  curiosity             211225 non-null  int64 
 11  desire                211225 non-null  int64 
 12  disappointment        211225 non-null  int64 
 13  disapproval           211225 non-null  int64 
 14  disgust               211225 non-null 

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [4]:
from skmultilearn.model_selection import iterative_train_test_split

# Filter out unclear examples
go_df = go_df[go_df["example_very_unclear"] == False]

# Extract emotion columns (from 'admiration' to 'neutral')
emotion_cols = go_df.columns[3:]
go_df = go_df[go_df[emotion_cols].sum(axis=1) > 0]  # remove empty labels

# Extract X (texts reshaped as array) and Y (multi-label binary matrix)
X = go_df["text"].values.reshape(-1, 1)
Y = go_df[emotion_cols].values

# Perform stratified multi-label split (10k train)
X_train, Y_train, X_test, Y_test = iterative_train_test_split(X, Y, test_size=1 - (10_000 / len(go_df)))

# Flatten and reconstruct
texts_go = X_train.ravel().tolist()
labels_go = Y_train

texts_go_test = X_test.ravel().tolist()
labels_go_test = Y_test

In [ ]:
label_counts = pd.DataFrame(labels_go, columns=emotion_cols).sum().sort_values()

print(label_counts)

grief               32
relief              62
pride               63
nervousness         87
embarrassment      119
remorse            122
fear               154
desire             184
disgust            255
surprise           265
excitement         271
caring             289
sadness            325
confusion          354
joy                384
anger              389
love               394
disappointment     408
optimism           419
realization        423
amusement          445
curiosity          466
disapproval        550
gratitude          559
annoyance          655
admiration         824
approval           848
neutral           2661
dtype: int64


In [6]:
sw_df["class"] = sw_df["class"].str.lower().str.strip()
sw_df = sw_df[sw_df["class"].isin(["suicide", "non-suicide"])]
sw_df["label"] = sw_df["class"].map({"non-suicide": 0, "suicide": 1})

# Training set
sw_df_sample, _ = train_test_split(
    sw_df, 
    train_size=10_000, 
    stratify=sw_df["label"].values,  #  keep label balance
    random_state=42
)
train_indices_sw = sw_df_sample.index

# Final train data
texts_sw = sw_df.loc[train_indices_sw, "text"].tolist()
labels_sw = sw_df.loc[train_indices_sw, "label"].values

# Final test data
sw_df_test = sw_df.drop(index=train_indices_sw).reset_index(drop=True)
texts_sw_test = sw_df_test["text"].tolist()
labels_sw_test = sw_df_test["label"].values

In [7]:
sw_df_sample["label"].value_counts()

label
1    5000
0    5000
Name: count, dtype: int64

# Custom Metrics

In [8]:
label_counts = np.sum(labels_go, axis=0) + 1e-6  # shape: [num_classes]
total_samples = labels_go.shape[0]

neg_counts = total_samples - label_counts
pos_weight = neg_counts / label_counts

class_weights = torch.tensor(pos_weight, dtype=torch.float)

In [9]:
def compute_metrics_sw(p):
    preds = p.predictions.argmax(axis=1)
    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
    
from sklearn.metrics import hamming_loss

def hamming_score(y_true, y_pred):
    # Sample-wise Jaccard
    acc_list = []
    for true, pred in zip(y_true, y_pred):
        if np.sum(true) == 0 and np.sum(pred) == 0:
            acc_list.append(1)
        else:
            acc_list.append(np.sum(np.logical_and(true, pred)) / np.sum(np.logical_or(true, pred)))
    return np.mean(acc_list)

def compute_metrics_goemo(p):
    labels = p.label_ids
    logits = torch.tensor(p.predictions)  # get logits from p.predictions
    probs = torch.sigmoid(logits)         # apply sigmoid
    preds = (probs > 0.5).int().numpy()   # threshold and convert to int

    return {
        'hamming_score': hamming_score(labels, preds),
        'f1_micro': f1_score(labels, preds, average='micro', zero_division=0),
        'f1_macro': f1_score(labels, preds, average='macro', zero_division=0),
        'precision_micro': precision_score(labels, preds, average='micro', zero_division=0),
        'recall_micro': recall_score(labels, preds, average='micro', zero_division=0),
    }

In [10]:
from torch.nn import BCEWithLogitsLoss

class CustomLossTrainer(Trainer):
    def __init__(self, *args, pos_weight=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=self.pos_weight.to(logits.device))
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

# Modeling GoEmotions

In [11]:
num_labels = len(emotion_cols)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels,
    problem_type="multi_label_classification"  # important for multi-label
)

model.to(device)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


class GoEmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": torch.tensor(self.labels[idx], dtype=torch.float)
        }


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts_go, labels_go, test_size=0.1, random_state=42
)

train_dataset = GoEmotionDataset(train_texts, train_labels, tokenizer)
eval_dataset = GoEmotionDataset(val_texts, val_labels, tokenizer)

train_args = TrainingArguments(
    output_dir="/kaggle/working/results_go",
    eval_strategy="epoch",       # evaluates each epoch
    logging_strategy="steps",          # this tells it to log!
    logging_steps=50,                  # log every 50 steps
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    logging_dir="/kaggle/working/logs_go",
    save_strategy="no",                # avoid saving every step
    disable_tqdm=False,                # ensure progress bars show up
    report_to="none"                   # disables wandb/comet if not used
)


trainer = CustomLossTrainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics_goemo,
    pos_weight=class_weights
)

trainer.train()
trainer.save_model("/kaggle/working/model_go")

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Hamming Score,F1 Micro,F1 Macro,Precision Micro,Recall Micro
1,1.045200,0.961347,0.161615,0.227076,0.212003,0.134174,0.738233
2,0.831500,0.958021,0.195421,0.252093,0.234412,0.153324,0.708505
3,0.724100,1.156674,0.229546,0.300143,0.259569,0.191630,0.691990
4,0.531800,1.290421,0.236821,0.302818,0.269119,0.198017,0.643270
5,0.391900,1.523509,0.255542,0.321966,0.281368,0.219970,0.600330
6,0.324300,1.839200,0.268627,0.329882,0.279575,0.235149,0.552436
7,0.271100,2.031066,0.292256,0.345469,0.290169,0.254702,0.536746
8,0.197800,2.379299,0.298338,0.348270,0.289481,0.265227,0.507019
9,0.182200,2.489447,0.312139,0.357552,0.292250,0.281576,0.489678
10,0.151100,2.582921,0.316838,0.362458,0.294613,0.289448,0.484723


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

# Modeling Suicide Watch 

In [13]:
class SuicideDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [14]:
model_sw = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

model_sw.to(device)

tokenizer_sw = AutoTokenizer.from_pretrained("distilbert-base-uncased")

train_texts_sw, val_texts_sw, train_labels_sw, val_labels_sw = train_test_split(
    texts_sw, labels_sw, test_size=0.1, random_state=42
)

train_dataset_sw = SuicideDataset(train_texts_sw, train_labels_sw, tokenizer)
eval_dataset_sw = SuicideDataset(val_texts_sw, val_labels_sw, tokenizer)

train_args_sw = TrainingArguments(
    output_dir="/kaggle/working/results_sw",
    eval_strategy="epoch",        # evaluates at end of each epoch
    logging_strategy="steps",           # log progress every few steps
    logging_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    logging_dir="/kaggle/working/logs_sw",
    save_strategy="no",
    disable_tqdm=False,
    report_to="none"
)

trainer_sw = Trainer(
    model=model_sw,
    args=train_args_sw,
    train_dataset=train_dataset_sw,
    eval_dataset=eval_dataset_sw,
    compute_metrics=compute_metrics_sw
)

trainer_sw.train()

trainer_sw.save_model("/kaggle/working/model_sw")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.120900,0.163879,0.951000,0.952102,0.938343,0.966270
2,0.094300,0.172685,0.955000,0.955135,0.959920,0.950397
3,0.013100,0.249064,0.952000,0.952941,0.941860,0.964286
4,0.000900,0.311805,0.952000,0.952569,0.948819,0.956349
5,0.000400,0.315781,0.951000,0.951533,0.948718,0.954365


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


# Model Testing

In [15]:
print("GoEmotions Validation Set Evaluation:")
if trainer is None:
    model_sw = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/model_go")
    trainer_sw = CustomLossTrainer(
        model=model,
        args=train_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics_goemo,
        pos_weight=class_weights
    )

# GoEmotions Evaluation
results_go = trainer.evaluate()
for key, value in results_go.items():
    print(f"{key}: {value:.4f}")


print("\nSuicide Watch Validation Set Evaluation:")
if trainer_sw is None:
    model_sw = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/model_sw")
    trainer_sw = Trainer(
        model=model_sw,
        args=train_args_sw,          # you must have this defined earlier
        train_dataset=train_dataset_sw,  # optional but good to include
        eval_dataset=eval_dataset_sw,           # we'll pass the test dataset manually
        compute_metrics=compute_metrics_sw
    )
    
# Suicide Watch Evaluation
results_sw = trainer_sw.evaluate()
for key, value in results_sw.items():
    print(f"{key}: {value:.4f}")

GoEmotions Validation Set Evaluation:


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


eval_loss: 2.5829
eval_hamming_score: 0.3168
eval_f1_micro: 0.3625
eval_f1_macro: 0.2946
eval_precision_micro: 0.2894
eval_recall_micro: 0.4847
eval_runtime: 3.4134
eval_samples_per_second: 292.3770
eval_steps_per_second: 18.4570
epoch: 10.0000

Suicide Watch Validation Set Evaluation:


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


eval_loss: 0.3158
eval_accuracy: 0.9510
eval_f1: 0.9515
eval_precision: 0.9487
eval_recall: 0.9544
eval_runtime: 5.2855
eval_samples_per_second: 189.1960
eval_steps_per_second: 11.9190
epoch: 5.0000


In [16]:
test_dataset_go = GoEmotionDataset(texts_go_test, labels_go_test, tokenizer)
test_dataset_sw = SuicideDataset(texts_sw_test, labels_sw_test, tokenizer)

# GoEmotions Evaluation
results_go = trainer.evaluate(eval_dataset=test_dataset_go)
print("GoEmotions Final Test Set Evaluation:")
for key, value in results_go.items():
    print(f"{key}: {value:.4f}")

# Suicide Watch Evaluation
results_sw = trainer_sw.evaluate(eval_dataset=test_dataset_sw)
print("\nSuicide Watch Final Test Set Evaluation:")
for key, value in results_sw.items():
    print(f"{key}: {value:.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


GoEmotions Final Test Set Evaluation:
eval_loss: 2.6362
eval_hamming_score: 0.3139
eval_f1_micro: 0.3572
eval_f1_macro: 0.3149
eval_precision_micro: 0.2874
eval_recall_micro: 0.4717
eval_runtime: 680.7528
eval_samples_per_second: 290.6140
eval_steps_per_second: 18.1640
epoch: 10.0000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Suicide Watch Final Test Set Evaluation:
eval_loss: 0.2440
eval_accuracy: 0.9610
eval_f1: 0.9611
eval_precision: 0.9579
eval_recall: 0.9643
eval_runtime: 1186.3498
eval_samples_per_second: 187.1910
eval_steps_per_second: 11.7000
epoch: 5.0000


In [17]:
import torch.nn.utils.prune as prune

# === Step 1: Load original FP32 model ===
model_path = "/kaggle/working/model_go"
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# === Step 2: Apply Unstructured L1 Pruning ===
prune_ratio = 0.3  # Prune 30% of weights

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name="weight", amount=prune_ratio)

# Remove pruning hooks (permanent zeroed weights)
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear) and hasattr(module, 'weight_orig'):
        prune.remove(module, 'weight')

# Optional: Save pruned-only model
torch.save(model, "/kaggle/working/model_go_pruned.pt")

# === Step 3: Apply Dynamic Quantization on Pruned Model ===
model_quant = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Save quantized model
torch.save(model_quant, "/kaggle/working/model_go_pruned_quantized.pt")


In [18]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch.nn.utils.prune as prune

# === Step 1: Load original FP32 model ===
model_path = "/kaggle/working/model_sw"
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# === Step 2: Apply Unstructured L1 Pruning ===
prune_ratio = 0.3  # Prune 30% of weights

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name="weight", amount=prune_ratio)

# Remove pruning hooks (permanent zeroed weights)
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear) and hasattr(module, 'weight_orig'):
        prune.remove(module, 'weight')

# Optional: Save pruned-only model
torch.save(model, "/kaggle/working/model_sw_pruned.pt")

# === Step 3: Apply Dynamic Quantization on Pruned Model ===
model_quant = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Save quantized model
torch.save(model_quant, "/kaggle/working/model_sw_pruned_quantized.pt")


In [19]:
print("You are finished in this project!")

You are finished in this project!
